# Positivity Check for ρ_W and Band-Limited Tests

This notebook:
1) plots the weight  
   \(\rho_W(x) = \frac{1}{2\pi}\left(\frac{1}{1+e^{-x/2}} + 2e^{-x/2}\right)\) for \(x>0\);
2) generates randomized nonnegative, **C^\u221e compactly supported** cosine densities \(g_b\) as sums of smooth bumps;
3) for each \(g_b\) computes

   \[ g(0) = \frac{1}{\pi} \int_0^\infty g_b(x)\,dx,\quad
      J(g) = \frac{1}{2\pi}\int_0^\infty e^{-x/2}\,g_b(x)\,dx,\quad
      A(g) = \frac{1}{2\pi}\int_0^\infty \frac{g_b(x)}{e^{x/2}+1}\,dx, \]

   verifies the identity
   \[ -A(g)+\tfrac12 g(0)+2J(g) = \int_0^\infty g_b(x)\,\rho_W(x)\,dx \ge 0, \]
   and reports min/mean margins \(\int g_b\,\rho_W\) across randomized trials.

**Test class & evenness.** We build \(g_b\in C_c^\infty(0,\infty)\) as sums of nonnegative smooth bumps; by cosine inversion,
\(g(\tau) = \frac{1}{\pi}\int_0^\infty g_b(x)\cos(x\tau)\,dx\) is even. We also check evenness numerically.

**Acceptance checks (printed at run-time):**
- Machine-precision agreement of the identity (LHS vs. RHS).
- Nonnegative margins across randomized trials (within tiny numerical tolerance).
- Trials span a range of support radii and shapes for \(g_b\).

### Core formulas implemented
- \(\rho_W(x)=\tfrac{1}{2\pi}\big(\tfrac{1}{1+e^{-x/2}}+2e^{-x/2}\big)\,1_{(0,\infty)}(x)\).
- \(g(0)=\tfrac{1}{\pi}\int_0^\infty g_b(x)\,dx\), \(J(g)=\tfrac{1}{2\pi}\int_0^\infty e^{-x/2}g_b(x)\,dx\),
  \(A(g)=\tfrac{1}{2\pi}\int_0^\infty \frac{g_b(x)}{e^{x/2}+1}\,dx\).
- Identity: \(-A(g)+\tfrac12 g(0)+2J(g)=\int_0^\infty g_b(x)\,\rho_W(x)\,dx\ge 0\).

In [ ]:
# Imports & reproducibility
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Callable, Tuple, List, Dict
import math, random

rng = np.random.default_rng(987654321)
random.seed(987654321)

# Plot defaults
plt.rcParams["figure.figsize"] = (6,4)
plt.rcParams["axes.grid"] = True

In [ ]:
def rho_W(x: np.ndarray) -> np.ndarray:
    """\n    \rho_W(x) = (1/(2\pi)) * ( 1/(1 + e^{-x/2}) + 2 e^{-x/2} ), for x>0; 0 at x<=0.\n    Vectorized for numpy arrays.\n    """
    x = np.asarray(x)
    out = np.zeros_like(x, dtype=float)
    pos = x > 0
    exph = np.exp(-x[pos] / 2.0)
    out[pos] = (1.0/(2.0*math.pi)) * ( 1.0/(1.0 + exph) + 2.0*exph )
    return out

def plot_rhoW(xmax: float = 20.0):
    x = np.linspace(0, xmax, 3000)
    y = rho_W(x)
    plt.figure()
    plt.plot(x, y, label=r"$\rho_W(x)$")
    plt.title(r"$\rho_W(x)=\frac{1}{2\pi}\left(\frac{1}{1+e^{-x/2}}+2e^{-x/2}\right)$ on $(0,\infty)$")
    plt.xlabel("x"); plt.ylabel(r"$\rho_W(x)$")
    plt.legend(); plt.show()

In [ ]:
def smooth_bump(x: np.ndarray, c: float, r: float) -> np.ndarray:
    """C^\infty bump: exp(-1/(1 - t^2)) for |t|<1, else 0. Support [c-r, c+r]."""
    t = (x - c) / r
    y = np.zeros_like(t, dtype=float)
    mask = np.abs(t) < 1.0
    y[mask] = np.exp(-1.0 / (1.0 - t[mask]**2))
    return y

@dataclass
class GBSpec:
    centers: List[float]
    radii: List[float]
    weights: List[float]
    support_max: float

def make_random_gb_spec(scale_min=0.75, scale_max=60.0) -> GBSpec:
    """Randomize gb by summing K nonnegative bumps with varied supports and centers."""
    log_s = rng.uniform(np.log(scale_min), np.log(scale_max))
    scale = float(np.exp(log_s))
    K = int(rng.integers(1, 6))  # 1..5 bumps
    centers, radii, weights = [], [], []
    for _ in range(K):
        c = float(rng.uniform(0.02*scale, 1.0*scale))   # positive centers
        r = float(rng.uniform(0.04*scale, 0.40*scale))  # widths
        if c - r < 0.0:
            c = r + float(rng.uniform(0.0, 0.1*scale))  # keep support > 0
        w = float(rng.uniform(0.2, 1.2))                # nonnegative weights
        centers.append(c); radii.append(r); weights.append(w)
    support_max = max(c + r for c, r in zip(centers, radii))
    return GBSpec(centers, radii, weights, 1.05*support_max)

def gb_from_spec(spec: GBSpec) -> Callable[[np.ndarray], np.ndarray]:
    """Return callable gb(x) built from GBSpec as nonnegative sum of C^\infty bumps."""
    def gb(x: np.ndarray) -> np.ndarray:
        x = np.asarray(x, dtype=float)
        val = np.zeros_like(x)
        for c, r, w in zip(spec.centers, spec.radii, spec.weights):
            val += w * smooth_bump(x, c, r)
        return val
    return gb

In [ ]:
def trapz_integral(x: np.ndarray, y: np.ndarray) -> float:
    return float(np.trapz(y, x))

def compute_functionals(x: np.ndarray, gb_vals: np.ndarray) -> Dict[str, float]:
    """Compute g(0), J(g), A(g), and \u222b gb\,\rho_W, from gb on grid x."""
    rho = rho_W(x)
    g0 = (1.0/math.pi) * trapz_integral(x, gb_vals)
    J  = (1.0/(2.0*math.pi)) * trapz_integral(x, np.exp(-x/2.0) * gb_vals)
    A  = (1.0/(2.0*math.pi)) * trapz_integral(x, gb_vals / (np.exp(x/2.0) + 1.0))
    rhs = trapz_integral(x, gb_vals * rho)
    lhs = -A + 0.5*g0 + 2.0*J
    abs_err = abs(lhs - rhs)
    rel_err = abs_err / max(1e-14, abs(rhs))
    return dict(g0=g0, J=J, A=A, lhs=lhs, rhs=rhs, abs_err=abs_err, rel_err=rel_err, margin=rhs)

def g_from_gb(tau: np.ndarray, x: np.ndarray, gb_vals: np.ndarray) -> np.ndarray:
    """Cosine inversion (numerical): g(\u03c4) = (1/\u03c0) \u222b_0^\u221e gb(x) cos(x\u03c4) dx."""
    # Composite trapezoid in x for each tau
    tau = np.asarray(tau, dtype=float)
    cos_mat = np.cos(np.outer(tau, x))
    integ = np.trapz(gb_vals * cos_mat, x, axis=1)
    return (1.0/math.pi) * integ

In [ ]:
def make_grid(xmax: float) -> np.ndarray:
    """Adaptive grid on [0, xmax] for stable trapezoid integration."""
    N = int(min(250000, max(4000, 240 * xmax)))
    if N % 2 == 0:
        N += 1
    return np.linspace(0.0, xmax, N)

def run_trials(n_trials: int = 60, store_examples: int = 4):
    rows = []
    curves: List[Tuple[np.ndarray, np.ndarray, GBSpec]] = []
    for t in range(1, n_trials+1):
        spec = make_random_gb_spec()
        gb = gb_from_spec(spec)
        x = make_grid(spec.support_max)
        gbx = gb(x)
        if len(curves) < store_examples:
            curves.append((x, gbx, spec))
        vals = compute_functionals(x, gbx)
        rows.append(dict(trial=t, K=len(spec.centers), support_max=spec.support_max, **vals))
    return pd.DataFrame(rows), curves

In [ ]:
def plot_gb_examples(curves: List[Tuple[np.ndarray, np.ndarray, GBSpec]]):
    for idx, (x, gbx, spec) in enumerate(curves, start=1):
        plt.figure()
        plt.plot(x, gbx, label=f"gb example #{idx} (K={len(spec.centers)})")
        plt.title(f"Sample gb(x) — compact support \u2264 {spec.support_max:.3f}")
        plt.xlabel("x"); plt.ylabel("gb(x)")
        plt.legend(); plt.show()

def plot_integrand_example(x: np.ndarray, gbx: np.ndarray):
    plt.figure()
    plt.plot(x, gbx * rho_W(x), label=r"$g_b(x)\,\rho_W(x)$")
    plt.title(r"Example integrand $g_b(x)\,\rho_W(x)$")
    plt.xlabel("x"); plt.ylabel("integrand")
    plt.legend(); plt.show()

## Run experiments and plots
We plot \(\rho_W\), generate randomized \(g_b\), verify the identity, check nonnegativity of margins, and inspect evenness of the recovered time-side \(g\).

In [ ]:
# Plot the weight
plot_rhoW(20.0)

# Run randomized trials
df, curves = run_trials(n_trials=60, store_examples=4)

# Equality and margin checks
abs_err_max = df['abs_err'].max()
rel_err_max = df['rel_err'].max()
margin_min  = df['margin'].min()
margin_mean = df['margin'].mean()

print("Equality check (lhs vs. \u222b gb\u22c5\u03c1_W):")
print(f"  max abs error  = {abs_err_max:.3e}")
print(f"  max rel error  = {rel_err_max:.3e}")
print("\nMargins (\u222b gb\u22c5\u03c1_W):")
print(f"  min margin     = {margin_min:.6e}")
print(f"  mean margin    = {margin_mean:.6e}")

# Assertions with small tolerances
TOL_EQ   = 1e-12
TOL_NEG  = -1e-10  # allow tiny negative from quadrature
assert abs_err_max < 1e-10 or rel_err_max < 1e-10, "Identity check failed beyond numerical tolerance."
assert margin_min >= TOL_NEG, "Found a negative margin beyond tolerance."

# Show first few rows
print("\nFirst 10 trial rows:\n")
display_cols = ['trial','K','support_max','g0','J','A','lhs','rhs','abs_err','rel_err','margin']
print(df[display_cols].head(10).to_string(index=False))

# Plot sample gb and an example integrand
plot_gb_examples(curves[:3])
plot_integrand_example(curves[0][0], curves[0][1])

# Evenness check for g(\u03c4) from the first example
x0, gb0, spec0 = curves[0]
tau = np.linspace(-10, 10, 801)
g_tau = g_from_gb(tau, x0, gb0)
even_err = np.max(np.abs(g_tau - g_tau[::-1]))
print(f"\nEvenness check for g(\u03c4) from cosine inversion: max |g(\u03c4)-g(-\u03c4)| = {even_err:.3e}")
assert even_err < 1e-8, "g(tau) is not numerically even within tolerance."

### Notes
- The nonnegativity of margins follows from \(g_b\ge 0\) and the strictly positive density \(\rho_W\) on \((0,\infty)\).
- The identity \(-A+\tfrac12 g(0)+2J=\int g_b\,\rho_W\) encodes the Mellin–torsion of the Poisson kernel and the \(e^{-x/2}\) drift.
- Grid resolution adapts to the support size to keep trapezoidal integration stable across narrow/wide bumps.